In [12]:
import rioxarray
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd

import dask

In [13]:
from dask.distributed import Client
client = Client()

/Users/marthamorrissey/mambaforge/envs/redwood-dev/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 64636 instead
  warnings.warn(


In [14]:
dset = rioxarray.open_rasterio('/Users/marthamorrissey/Downloads/simple_dist_stack_Southern_Rockies.tif',
                               band_as_variable=True)

In [15]:
co_boundary = gpd.read_file('/Users/marthamorrissey/Downloads/Colorado_State_Boundary.geojson')

In [16]:
co_boundary_reproject = co_boundary.to_crs(dset.rio.crs)

In [17]:
dset_clipped = dset.rio.clip_box(*co_boundary_reproject.total_bounds)

In [6]:
dset_clipped.nbytes / 1e9

5.617235344

In [6]:
np.unique(dset['band_17'])

array([  0,   1,   2, 255], dtype=uint8)

In [18]:
years = range(1999, 2021)
time_coords = pd.to_datetime([f'{year}-01-01' for year in years])  # Convert years to datetime objects

In [19]:
len(time_coords)

22

In [20]:
time_lst = []

for i, v in enumerate(dset.data_vars):
    dset_tmp = dset[v].to_dataset().assign_coords(time=time_coords[i])
    dset_tmp = dset_tmp.rename({list(dset_tmp.data_vars)[0] : 'dist'})
    dset_tmp = dset_tmp.chunk(chunks="auto")
    time_lst.append(dset_tmp)

In [22]:
dset_format = xr.concat(time_lst, dim='time')

In [25]:
dset_format['dist'][0]

<xarray.DataArray 'dist' (y: 28057, x: 16339)>
dask.array<getitem, shape=(28057, 16339), dtype=uint8, chunksize=(11585, 11585), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 -1.183e+06 -1.183e+06 ... -6.932e+05 -6.932e+05
  * y            (y) float64 2.243e+06 2.243e+06 ... 1.402e+06 1.402e+06
    spatial_ref  int64 0
    time         datetime64[ns] 1999-01-01
Attributes:
    STATISTICS_MAXIMUM:  1
    STATISTICS_MEAN:     -9999
    STATISTICS_MINIMUM:  0
    STATISTICS_STDDEV:   -9999
    _FillValue:          255
    scale_factor:        1.0
    add_offset:          0.0
    long_name:           forest-disturbance1999